In [ ]:
(in-package "ACL2")

In [32]:
(include-book "chat-lib"
              :ttags ((:quicklisp) (:quicklisp.osicat) (:quicklisp.dexador) 
                      (:http-json) (:llm-client) (:mcp-client)))


You: Show a few theorems about natural number arithmetics
 "ACL2"

The event ( INCLUDE-BOOK "chat-lib" ...) is redundant.  See :DOC redundant-
events.

Summary
Form:  ( INCLUDE-BOOK "chat-lib" ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 :REDUNDANT


In [33]:
;; Enable redefinition so *chat-agent* can be updated between turns
(set-ld-redefinition-action '(:doit . :overwrite) state)

 (:DOIT . :OVERWRITE)


In [34]:
(make-event
 (mv-let (err models state)
   (llm-list-models-full state)
   (declare (ignorable err))
   (let ((selected (select-completions-model models *model-prefs*)))
     (prog2$
      (if selected
          (cw "~%Model: ~s0 (~x1 tokens)~%"
              (model-info->id selected)
              (model-info->loaded-context-length selected))
        (cw "~%WARNING: No model found. Ensure LM Studio is running.~%"))
      (mv nil 
          `(defconst *chat-model* 
             ,(if selected (model-info->id selected) "no-model-found"))
          state)))))


Model: qwen/qwen3-coder-30b (80000 tokens)

The event ( DEFCONST *CHAT-MODEL* ...) is redundant.  See :DOC redundant-
events.

Summary
Form:  ( DEFCONST *CHAT-MODEL* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)

Summary
Form:  ( MAKE-EVENT (MV-LET ...))
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 :REDUNDANT


In [35]:
;; Initialize fresh agent state
(defconst *initial-chat-state*
  (init-agent-conversation *default-system-prompt* *default-agent-config*))


The event ( DEFCONST *INITIAL-CHAT-STATE* ...) is redundant.  See :DOC
redundant-events.

Summary
Form:  ( DEFCONST *INITIAL-CHAT-STATE* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 :REDUNDANT


In [36]:
(make-event
 (mv-let (mcp-err mcp-conn state)
   (mcp-connect *mcp-default-endpoint* state)
   (prog2$
    (if mcp-err
        (cw "~%MCP connection failed: ~s0~%Code execution disabled.~%" mcp-err)
      (prog2$
       (cw "~%MCP connected.~%")
       (if (mcp-connection-has-acl2-session-p mcp-conn)
           (cw "ACL2 session: ~s0~%" (mcp-connection->acl2-session-id mcp-conn))
         (cw "No persistent ACL2 session.~%"))))
    (mv nil
        `(defconst *chat-mcp* ',mcp-conn)
        state))))


MCP connected.
ACL2 session: 06b13c99-60d2-482a-a602-28d6ccd1c4ba

Summary
Form:  ( DEFCONST *CHAT-MCP* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)

Summary
Form:  ( MAKE-EVENT (MV-LET ...))
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *CHAT-MCP*


In [37]:
;; Agent state starts fresh - we use a defconst that gets redefined
(defconst *chat-agent* *initial-chat-state*)


Summary
Form:  ( DEFCONST *CHAT-AGENT* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *CHAT-AGENT*


In [38]:
;; The main chat function - returns new state but also redefines *chat-agent*
(defun chat-fn (msg state)
  "Send a message and get a response. Updates *chat-agent* state."
  (declare (xargs :mode :program :stobjs state))
  (prog2$ (cw "~%You: ~s0~%" msg)
    (mv-let (new-agent state)
      (chat-turn msg *chat-agent* *chat-model* *chat-mcp* state)
      (mv nil
          `(defconst *chat-agent* ',new-agent)
          state))))


The event ( DEFUN CHAT-FN ...) is redundant.  See :DOC redundant-events.

Summary
Form:  ( DEFUN CHAT-FN ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 :REDUNDANT


In [39]:
;; User-facing macro for simple invocation
(defmacro chat (msg)
  "Send a message to the chat agent. Example: (chat \"Hello!\")"
  `(make-event (chat-fn ,msg state)))


The event ( DEFMACRO CHAT ...) is redundant.  See :DOC redundant-events.

Summary
Form:  ( DEFMACRO CHAT ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 :REDUNDANT


In [40]:
;; Reset conversation to initial state
(defmacro chat-reset ()
  "Reset the conversation to initial state."
  '(progn
     (defconst *chat-agent* *initial-chat-state*)
     (value-triple :chat-reset)))


The event ( DEFMACRO CHAT-RESET ...) is redundant.  See :DOC redundant-
events.

Summary
Form:  ( DEFMACRO CHAT-RESET ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 :REDUNDANT


In [41]:
;; Show current conversation
(defmacro chat-history ()
  "Display the current conversation history."
  '(value-triple (show-conversation *chat-agent*)))


The event ( DEFMACRO CHAT-HISTORY ...) is redundant.  See :DOC redundant-
events.

Summary
Form:  ( DEFMACRO CHAT-HISTORY ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 :REDUNDANT


In [42]:
;; Show context usage
(defmacro chat-usage ()
  "Display context token usage."
  '(value-triple (show-context-usage *chat-agent*)))


The event ( DEFMACRO CHAT-USAGE ...) is redundant.  See :DOC redundant-
events.

Summary
Form:  ( DEFMACRO CHAT-USAGE ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 :REDUNDANT


In [43]:
(defconst *chat-ready*
  (prog2$ (cw "~%========================================~%")
    (prog2$ (cw "Chat Ready!~%")
      (prog2$ (cw "========================================~%")
        (prog2$ (cw "~%Commands:~%")
          (prog2$ (cw "  (chat \"your message\")  - Send a message~%")
            (prog2$ (cw "  (chat-reset)           - Start fresh~%")
              (prog2$ (cw "  (chat-history)         - Show conversation~%")
                (prog2$ (cw "  (chat-usage)           - Show token usage~%")
                  (cw "~%"))))))))))


Chat Ready!

Commands:
  (chat "your message")  - Send a message
  (chat-reset)           - Start fresh
  (chat-history)         - Show conversation
  (chat-usage)           - Show token usage


The event ( DEFCONST *CHAT-READY* ...) is redundant.  See :DOC redundant-
events.

Summary
Form:  ( DEFCONST *CHAT-READY* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 :REDUNDANT


In [44]:
(chat "Hello! You are you?  What can you do?")


You: Hello! You are you?  What can you do?

Assistant: Hello! I am an AI assistant running inside a formally verified
ReAct agent framework built in ACL2. I'm designed to help with reasoning
and problem solving.

I can:
- Execute ACL2 code snippets
- Help with formal verification tasks
- Assist with logical reasoning
- Solve mathematical problems
- Explain ACL2 concepts and code

I can run ACL2 code when you provide it in fenced code blocks. For
example:

```acl2
(+ 1 2 3)
```

What would you like me to help you with today?

[Executing: (+ 1 2 3)]

Result: NIL
ACL2 !>6
ACL2 !>

Assistant: I notice the ACL2 environment is showing some unexpected
output with "NIL" and "ACL2 !>6" followed by "ACL2 !>". This suggests
there might have been an incomplete command or error in the previous
interaction.

Let me try a simple ACL2 command to demonstrate functionality:

```acl2
(+ 1 2 3)
```

This should return 6, showing that the ACL2 environment is working
properly. 

Is there something specific

In [45]:
(chat "Show a few theorems about natural number arithmetics")


You: Show a few theorems about natural number arithmetics

Assistant: Let me demonstrate a few basic theorems about natural number
arithmetic in ACL2:

```acl2
(defthm commutativity-of-addition
  (equal (+ x y) (+ y x))
  :rule-classes :commutativity)

(defthm associativity-of-addition
  (equal (+ (+ x y) z) (+ x (+ y z)))
  :rule-classes :associativity)

(defthm identity-of-addition
  (equal (+ x 0) x)
  :rule-classes :identity)

(defthm commutativity-of-multiplication
  (equal (* x y) (* y x))
  :rule-classes :commutativity)

(defthm distributivity-of-multiplication-over-addition
  (equal (* x (+ y z)) (+ (* x y) (* x z)))
  :rule-classes :distributivity)
```

These are fundamental theorems about natural number arithmetic that
would be proven in ACL2. The `:rule-classes` provide hints to the theorem
prover about how these theorems should be used as rewriting rules.

Would you like me to prove any of these theorems or show more examples?

[Executing: (defthm commutativity-of-addition